In [4]:
import pandas as pd
import numpy as np
import time

In [7]:
train_df = pd.read_csv("/opt/ml/input/data/train/train_ratings.csv")
conv_year = list()
for t in train_df['time']:
    y, m = map(int, time.strftime('%Y-%m', time.localtime(t)).split("-"))
    conv_year.append(y)
train_df['year'] = conv_year

In [21]:
train_df

,user,item,time,year
0,11,4643,1230782529,2009
1,11,170,1230782534,2009
2,11,531,1230782539,2009
3,11,616,1230782542,2009
4,11,2140,1230782563,2009
...,...,...,...,...
5154466,138493,44022,1260209449,2009
5154467,138493,4958,1260209482,2009
5154468,138493,68319,1260209720,2009
5154469,138493,40819,1260209726,2009


In [18]:
index=sorted(train_df['year'].unique())
col = train_df['item'].unique()

,4643,170,531,616,2140,2722,2313,2688,2428,3113,...,6478,85259,842,5999,6864,7753,93422,6519,8830,102880
2005,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2006,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2007,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2008,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2009,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2010,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2011,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2012,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2013,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2014,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
from tqdm import tqdm

count_mov = pd.DataFrame(index=index, columns=col, data=0)

for item, year in zip(tqdm(train_df['item']), train_df['year']):
    count_mov.loc[year,item] += 1

100%|██████████| 5154471/5154471 [19:06<00:00, 4494.21it/s]


In [25]:
count_mov

,4643,170,531,616,2140,2722,2313,2688,2428,3113,...,6478,85259,842,5999,6864,7753,93422,6519,8830,102880
2005,139,125,100,95,224,68,289,56,74,44,...,15,0,12,7,19,4,0,6,2,0
2006,152,118,110,107,182,65,282,67,79,48,...,11,0,11,3,14,12,0,9,8,0
2007,138,134,142,97,166,59,252,94,65,48,...,4,0,4,4,12,8,0,8,6,0
2008,152,184,148,153,205,73,257,83,82,58,...,8,0,6,5,4,5,0,6,6,0
2009,106,141,120,141,190,67,206,43,65,55,...,4,0,7,8,6,2,0,4,2,0
2010,101,112,92,122,146,57,181,19,65,41,...,4,0,2,3,9,4,0,6,2,0
2011,85,78,78,117,114,47,157,54,38,31,...,2,27,2,5,5,5,0,2,5,0
2012,86,98,68,109,133,41,189,49,41,34,...,6,15,2,5,5,1,2,1,5,0
2013,69,65,56,72,81,32,150,34,51,28,...,2,5,3,2,1,1,24,0,1,23
2014,57,39,37,53,55,24,126,7,29,11,...,1,11,1,1,1,0,16,0,2,21


In [41]:
uniq_user = train_df['user'].unique()
answer_list = list()
for user in tqdm(uniq_user):
    user_time_list = train_df[train_df['user'] == user]['year']
    user_seen_list = train_df[train_df['user'] == user]['item'].unique()
    max_year = max(user_time_list) # 끝 년도
    min_year = min(user_time_list) - 1 # 시작 년도
    # print ("min_year ", min_year)
    # print ("max_year", max_year)
    
    year = range(min_year, max_year + 1)
    movie_count_by_year = count_mov.loc[min_year:max_year].sum(axis=0)
    sorted_movies = movie_count_by_year.sort_values(ascending=False)
    
    cnt = 0
    for item in sorted_movies.index:
        if item not in user_seen_list:
            answer_list.append([user, item])
            cnt += 1
            if cnt >= 10:
                break

answer_list
    

100%|██████████| 31360/31360 [07:37<00:00, 68.48it/s]


[[11, 47],
 [11, 50],
 [11, 858],
 [11, 2329],
 [11, 56367],
 [11, 55820],
 [11, 5418],
 [11, 4963],
 [11, 1089],
 [11, 54286],
 [14, 2571],
 [14, 296],
 [14, 2959],
 [14, 318],
 [14, 4993],
 [14, 2858],
 [14, 7153],
 [14, 5952],
 [14, 593],
 [14, 48516],
 [18, 2571],
 [18, 2959],
 [18, 296],
 [18, 4993],
 [18, 7153],
 [18, 5952],
 [18, 4226],
 [18, 260],
 [18, 1198],
 [18, 1196],
 [25, 58559],
 [25, 60069],
 [25, 7153],
 [25, 4226],
 [25, 68157],
 [25, 47],
 [25, 260],
 [25, 1196],
 [25, 68358],
 [25, 1270],
 [31, 79132],
 [31, 318],
 [31, 2959],
 [31, 296],
 [31, 109374],
 [31, 356],
 [31, 99114],
 [31, 593],
 [31, 106782],
 [31, 4226],
 [35, 2959],
 [35, 5952],
 [35, 2762],
 [35, 1198],
 [35, 260],
 [35, 1196],
 [35, 4306],
 [35, 33794],
 [35, 1210],
 [35, 1136],
 [43, 58559],
 [43, 4993],
 [43, 7153],
 [43, 2858],
 [43, 5952],
 [43, 593],
 [43, 4226],
 [43, 56367],
 [43, 55820],
 [43, 59315],
 [50, 4993],
 [50, 1198],
 [50, 260],
 [50, 1196],
 [50, 4226],
 [50, 50],
 [50, 33794],
 

In [43]:
pd.DataFrame(answer_list, columns=['user', 'item']).to_csv("./submission_by_year.csv", index=False)